In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import os
import subprocess
from modis_scraper import MODIS_Scraper

In [1]:
# Downloading and pre-processing NVDI data from USGS MODIS website 
# Converting HDF files into netCDF format (.nc)
# Crop and re-project data to the same grid as mask.nc

In [14]:
products = ['MOLT/MOD13C1.006/', 'MOLA/MYD13C1.006/']
f_path = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/nvdi/nc/'

for product in products:
    m = MODIS_Scraper(mask='/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/mask.nc')
    dates = m.get_dates(product, tiled=False, convert=True)
    m.get_data(f_path, format='hdf')


In [6]:
f_path = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/nvdi/nc/'
files = os.listdir(f_path)

def cdo_merge(f1, f2):
    subprocess.call(['cdo', 'merge', f_path+f1, f_path+f2, f_path+f2+'.out.nc'])
    return f2+'.out.nc'

reduce(cdo_merge, files)

# In command line...
# ncap2 -s "Band1=float(Band1)" all.ndvi.nc all.float.ndvi.nc
# cdo divc,10000 all.float.ndvi.nc all.ndvi.01.nc
# cdo timmax all.ndvi.01.nc all.ndvi.max.nc
# cdo div all.ndvi.01.nc all.ndvi.max.nc all.ndvi.normed.nc

'2011.12.27.mask.nc.out.nc.out.nc'

In [ ]:
# Getting gridded weather data from UW server
# Reference: http://www.hydro.washington.edu/SurfaceWaterGroup/Data/livneh/livneh.et.al.2013.page.html
# Citation: Livneh, B., E. A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K. M. Andreadis, E. P. Maurer, and 
#     D. P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the 
#    Conterminous United States: Update and Extensions. J . Climate, 26.

# Version: ftp://ftp.hydro.washington.edu/pub/blivneh/CONUS/Meteorology.nc.v.1.2.1915.2011.bz2/VERSION_ID
#Version 1.2
#March, 2014
#ben.livneh@colorado.edu

In [13]:
%%writefile get_livneh_weather.py
import numpy as np
import subprocess

base_url = 'ftp://ftp.hydro.washington.edu/pub/blivneh/CONUS/Meteorology.nc.v.1.2.1915.2011.bz2/'
base_fname = 'Meteorology_Livneh_CONUSExt_v.1.2_2013.{t}.nc.bz2'

f_path = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/weather/'
mask = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/final/mask.nc'

yrs = np.arange(2004, 2012)
months = np.arange(1, 13)
variables = ['Prec', 'Tmax', 'Tmin', 'Wind']

for yr in yrs:
    for month in months:
        time = '{yr}{month:02d}'.format(yr=yr, month=month)
        fname = f_path + time + '{mod}{ext}'
        print 'Downloading for', time
        # Obtaining compressed file from UW ftp server
        subprocess.call(['wget', '-O', fname.format(mod='', ext='.nc.bz2'), base_url + base_fname.format(t=time)])
        # Unzip the file
        subprocess.call(['bunzip2', fname.format(mod='', ext='.nc.bz2')])
        
        print 'Processing for', time
        # Convert float data into integer, change data type into short to reduce file size (by half!)
        subprocess.call(['cdo', 'int', fname.format(mod='', ext='.nc'), fname.format(mod='.int', ext='.nc')])
        subprocess.call(['ncap2', '-s', 'Prec=short(Prec);Tmax=short(Tmax);Tmin=short(Tmin);Wind=short(Wind)',\
                        fname.format(mod='.int', ext='.nc'), fname.format(mod='.short', ext='.nc')])
        
        # Subsetting out each variable since different remap functions are being used
        for var in variables:
            subprocess.call(['cdo', 'selname,{var}'.format(var=var), \
                            fname.format(mod='.short', ext='.nc'), fname.format(mod='.short.'+str(var), ext='.nc')])
        subprocess.call(['cdo', 'remapcon,{grid}'.format(grid=mask), \
                        fname.format(mod='.short.Prec', ext='.nc'), fname.format(mod='.Prec', ext='.nc')])
        subprocess.call(['cdo', 'remapbil,{grid}'.format(grid=mask), \
                        fname.format(mod='.short.Tmax', ext='.nc'), fname.format(mod='.Tmax', ext='.nc')])
        subprocess.call(['cdo', 'remapbil,{grid}'.format(grid=mask), \
                        fname.format(mod='.short.Tmin', ext='.nc'), fname.format(mod='.Tmin', ext='.nc')])
        subprocess.call(['cdo', 'remapbil,{grid}'.format(grid=mask), \
                        fname.format(mod='.short.Wind', ext='.nc'), fname.format(mod='.Wind', ext='.nc')])
        
        # Clean up the directories
        subprocess.call(['rm', fname.format(mod='', ext='.nc'), fname.format(mod='.int', ext='.nc'), \
                        fname.format(mod='.short', ext='.nc')])
        for var in variables:
            subprocess.call(['rm', fname.format(mod='.short.'+str(var), ext='.nc')])

    print 'Merging year', yr
    for var in variables:
        file_list = [f_path+'{yr}{month:02d}.{var}.nc'.format(yr=yr, month=month, var=var) for month in months]
        subprocess.call(['cdo', 'mergetime', ' '.join(file_list), f_path+'{yr}.{var}.nc'.format(yr=yr, var=var)])
        for f in file_list:
            subprocess.call(['rm', f])


Overwriting get_livneh_weather.py


In [14]:
%run get_livneh_weather.py

Processing for 200401
Processing for 200402
Processing for 200403
Processing for 200404
Processing for 200405
Processing for 200406
Processing for 200407
Processing for 200408
Processing for 200409
Processing for 200410
Processing for 200411
Processing for 200412
Merging year 2004
Processing for 200501
Processing for 200502
Processing for 200503
Processing for 200504
Processing for 200505
Processing for 200506
Processing for 200507
Processing for 200508
Processing for 200509
Processing for 200510
Processing for 200511
Processing for 200512
Merging year 2005
Processing for 200601
Processing for 200602
Processing for 200603
Processing for 200604
Processing for 200605
Processing for 200606
Processing for 200607
Processing for 200608
Processing for 200609
Processing for 200610
Processing for 200611
Processing for 200612
Merging year 2006
Processing for 200701
Processing for 200702
Processing for 200703
Processing for 200704
Processing for 200705
Processing for 200706
Processing for 200707


In [3]:
# Getting historic climate normals from PRISM project
# http://www.prism.oregonstate.edu/documents/PRISM_datasets.pdf
# http://www.prism.oregonstate.edu/normals/

In [26]:
%%writefile process_prism.py
# Obtained historic climate normals from PRISM project
# http://www.prism.oregonstate.edu/documents/PRISM_datasets.pdf
# http://www.prism.oregonstate.edu/normals/

import subprocess
import numpy as np

variables = ['ppt', 'tmean']
folder = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/normals/'
base_fname = 'PRISM_{var}_30yr_normal_4kmM2_all_asc/PRISM_{var}_30yr_normal_4kmM2_{m:02d}_asc.asc'
grid = '/Users/Chiao/google-drive/projects/Galvanize/fall-foliage-finder/data/mask.nc'
base_nc_name = '{var}.{m:02d}.nc'
gdal = '/opt/local/bin/gdal_translate'

months = np.arange(1, 13)

for var in variables:
    files = []
    for month in months:
        subprocess.call([gdal, '-of', 'netCDF', 
                         folder+base_fname.format(var=var, m=month), 
                         folder+base_nc_name.format(var=var, m=month)])
        files.append(folder+base_nc_name.format(var=var, m=month))
    subprocess.call(['cdo', 'merge', ' '.join(files), folder+'{var}.monthly.nc'.format(var=var)])
    for f in files:
        subprocess.call(['rm', f])

subprocess.call(['cdo', 'remapcon,{grid}'.format(grid=grid), 
                    folder+'ppt.monthly.nc', folder+'ppt.monthly.mask.nc'])

subprocess.call(['cdo', 'remapbil,{grid}'.format(grid=grid), 
                    folder+'tmean.monthly.nc', folder+ 'tmean.monthly.mask.nc'])

Overwriting process_prism.py


In [27]:
%run process_prism.py